In [45]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import openpyxl
from typing import Tuple

In [129]:
DIMENSIONS: int = 10
BITS_PER_DIMENSION: int = 16
DOMAIN_1: tuple[float, float] = (-3, 3)
DOMAIN_2: tuple[float, float] = (-32.768, 32.768)
EVAL_LIMIT: int = 10000
REPEATS: int = 100
POPULATION_SIZE: int = 20

# Generate populations

In [130]:
def generate_initial_population_binary() -> np.ndarray:
    """Generates an initial binary population."""
    return np.array([
        ''.join(np.random.choice(['0', '1'], size=BITS_PER_DIMENSION)) 
        for _ in range(POPULATION_SIZE)
    ])

In [172]:
def generate_initial_population_real(domain: tuple[float, float]) -> np.ndarray:
    """Generates an initial real-valued solution for Simulated Annealing within the domain.

    Parameters:
    - domain (tuple[float, float]): Lower and upper bounds of the solution space.

    Returns:
    - np.ndarray: Array of real values (dtype=float) within the specified domain.
    """
    min_val, max_val = domain
    return np.random.uniform(min_val, max_val, (POPULATION_SIZE, DIMENSIONS)).astype(float)

In [173]:
population = generate_initial_population_real(DOMAIN_1)
population

array([[-2.87790536, -1.65106729, -0.03341029, -1.34210503, -0.4201863 ,
         2.79942509,  1.813128  , -0.15403636, -2.46501826, -1.81178924],
       [ 2.12041219,  0.29116235,  2.39033226,  1.00881605,  0.13725601,
        -2.63233905, -2.33063847,  1.70632294, -2.88790411,  2.66457705],
       [-2.1573493 , -1.99797211,  1.21402775, -0.38022002,  2.00795399,
         1.67519593,  2.80651393, -1.10340248, -0.33504429, -0.63681504],
       [ 1.8464206 , -1.44490104, -1.11637269, -1.35009342, -2.97962559,
         1.27779274,  1.74785919, -1.49020784, -1.50534928,  0.46021629],
       [ 0.76156069,  1.63586231,  2.23648305, -2.52749063, -0.82228459,
        -1.24268991,  2.63313708,  0.21622211, -2.8595153 , -2.37724648],
       [-2.89536943, -2.25580027, -0.34239295, -0.67487343, -1.51598004,
         1.44003985,  0.10549107,  0.66283636,  0.76242224,  0.08080782],
       [ 0.71353528,  2.08976779, -2.19844387, -1.4333529 , -1.05680338,
        -1.95473958, -1.9781425 , -2.953038  

# Eval functions

In [174]:
def evaluate_f1(solution: np.ndarray, domain: tuple[float, float]) -> float:
    """Evaluates the first function (F1) on a given solution.
    
    Parameters:
    - solution (np.ndarray): Solution array in either binary or real format.
    - domain (tuple[float, float]): Domain bounds.

    Returns:
    - float: Calculated function value.
    """
    sum_solution = np.sum(solution ** 2)
    exp_val = -5 / (1 + sum_solution)
    y = exp_val + np.sin(1 / np.tan(np.exp(exp_val)))
    return y

In [175]:
evaluate_f1(population, DOMAIN_1)

0.5999206477200943

In [136]:
def evaluate_f2(solution: np.ndarray, domain: tuple[float, float], a: float = 20, b: float = 0.2, c: float = 2 * np.pi) -> float:
    """Evaluates the second function (F2) on a given solution.

    Parameters:
    - solution (np.ndarray): Solution array in either binary or real format.
    - domain (tuple[float, float]): Domain bounds.

    Returns:
    - float: Calculated function value.
    """
    d = len(solution)
    sum_1 = np.sum(solution ** 2)
    sum_2 = np.sum(np.cos(c * solution))
    return -a * np.exp(-b * np.sqrt(sum_1 / d)) - np.exp(sum_2 / d) + a + np.exp(1)

# Tournament Selection

In [137]:
def tournament_selection(population: np.ndarray, fitness: np.ndarray, k: int = 2) -> str:
    """Performs tournament selection on a binary population."""
    indices = np.random.choice(len(population), size=k, replace=False)
    selected = population[indices]
    selected_fitness = fitness[indices]
    return selected[np.argmax(selected_fitness)]

In [179]:
fitness_values = np.array([evaluate_f1(ind, (0, 10)) for ind in population])
parent_1 = tournament_selection(population, fitness_values)
parent_2 = tournament_selection(population, fitness_values)
parent_1, parent_2

(array([ 0.71353528,  2.08976779, -2.19844387, -1.4333529 , -1.05680338,
        -1.95473958, -1.9781425 , -2.953038  ,  0.82182471, -0.54172093]),
 array([-0.17326325,  2.65459667, -2.02027722,  0.80200964, -0.35161143,
        -2.48265957,  1.36077191, -2.19942072, -1.59076119, -2.66746255]))

# Recombination

In [132]:
def uniform_crossover_binary(parent1: str, parent2: str) -> str:
    """Performs uniform crossover on two binary strings."""
    mask = np.random.choice([0, 1], size=BITS_PER_DIMENSION).astype(bool)
    offspring = ''.join(
        p1 if m else p2 for p1, p2, m in zip(parent1, parent2, mask)
    )
    return offspring

In [160]:
def discrete_recombination(parent1: np.ndarray, parent2: np.ndarray) -> np.ndarray:
    """
    Performs discrete recombination on two real-valued vectors.

    Parameters:
    - parent1 (np.ndarray): Real-valued vector of parent 1.
    - parent2 (np.ndarray): Real-valued vector of parent 2.

    Returns:
    - np.ndarray: Offspring resulting from recombination.
    """
    mask = np.random.rand(DIMENSIONS) > 0.5  # Random mask
    return np.where(mask, parent1, parent2) 

In [180]:
child = discrete_recombination(parent_1, parent_2)
child

array([ 0.71353528,  2.65459667, -2.19844387, -1.4333529 , -0.35161143,
       -2.48265957, -1.9781425 , -2.19942072, -1.59076119, -2.66746255])

# Mutations

In [133]:
def binary_mutation(solution: str, mutation_rate: float = 0.1) -> str:
    """Mutates a binary string."""
    mask = np.random.rand(len(solution)) < mutation_rate
    mutated_solution = ''.join(
        '1' if (bit == '0' and m) else '0' if (bit == '1' and m) else bit
        for bit, m in zip(solution, mask)
    )
    return mutated_solution

In [103]:
def real_mutation(solution: np.ndarray, sigma: float) -> np.ndarray:
    """
    Mutates a real-valued vector with uncorrelated mutation.

    Parameters:
    - individual (np.ndarray): Real-valued vector to mutate.
    - sigma (float): Current mutation step size.

    Returns:
    - Tuple[np.ndarray, float]: Mutated vector and new mutation step size.
    """
    tau = 1 / np.sqrt(DIMENSIONS)
    sigma_prime = sigma * np.exp(tau * np.random.normal())
    mutated_solution = solution + np.random.normal(0, sigma_prime, DIMENSIONS)
    return np.clip(mutated_solution, DOMAIN_1[0], DOMAIN_1[1]) 

In [183]:
mutant = real_mutation(child, sigma=1)
mutant

array([ 1.45568824,  2.65413407, -0.74261431, -2.56838799, -1.53645288,
       -1.41949611, -2.57166911, -2.50792178, -2.17755796, -3.        ])

# Util Functions

In [184]:
def decode_solution_binary(population: np.ndarray, domain: tuple[float, float]) -> np.ndarray:
    """Decodes binary solutions into real-valued vectors."""
    min_val, max_val = domain
    scale = (max_val - min_val) / (2**BITS_PER_DIMENSION - 1)
    decoded = []
    for individual in population:
        dimensions = [
            int(individual[i:i+BITS_PER_DIMENSION], 2) * scale + min_val
            for i in range(0, len(individual), BITS_PER_DIMENSION)
        ]
        decoded.append(dimensions)
    return np.array(decoded)

# Run Experiment

In [188]:
def genetic_algorithm():
    evaluations = 0
    population = generate_initial_population_real(DOMAIN_1)
    fitness_values = np.array([evaluate_f1(ind, (0, 10)) for ind in population])
    evaluations += len(population)

    while evaluations < EVAL_LIMIT:
        children = []
        
        for _ in range(0, POPULATION_SIZE, 2):
            parent_1 = tournament_selection(population, fitness_values)
            parent_2 = tournament_selection(population, fitness_values)
    
            child_1 = discrete_recombination(parent_1, parent_2)
            child_2 = discrete_recombination(parent_1, parent_2)
            mutant_1 = real_mutation(child_1, sigma=1)
            mutant_2 = real_mutation(child_2, sigma=1)
        
            children.append(mutant_1)
            children.append(mutant_2)

        children = np.array(children)
        children_fitness = children_fitness = np.array([evaluate_f1(ind, domain) for ind in children])
        evaluations += len(population)
        

        fitness_values = np.array([evaluate_f1(ind, (0, 10)) for ind in population])

    return children

pd.DataFrame(genetic_algorithm())

,0,1,2,3,4,5,6,7,8,9
0,1.114157,-3.000000,-3.000000,3.000000,2.111434,0.645478,-0.420295,-0.945200,0.996166,-3.000000
1,-2.836838,-1.424327,-0.129400,2.384067,2.433295,0.738171,0.329213,-2.120805,2.054296,-2.592301
2,-0.644906,-3.000000,-2.293135,3.000000,1.285176,-2.053265,-2.229401,-0.935167,-1.369895,-1.629598
3,-2.169096,-2.308682,-2.880874,2.673814,-0.196741,-1.236743,1.146991,-1.377851,0.103790,0.687124
4,0.913270,1.628598,-1.781261,0.296913,-1.165561,-2.141666,-3.000000,-0.945544,-1.974938,-0.904670
5,-0.340353,-1.337477,-1.294828,0.608165,-0.684383,-1.943786,-3.000000,-0.082426,-2.600037,-1.786934
6,-0.933842,-1.625350,3.000000,2.240439,-2.604615,-0.770131,1.069608,2.551487,-0.603649,1.443538
7,1.467467,-2.247255,2.223863,-2.910659,0.932892,1.023182,-3.000000,-3.000000,-0.188302,0.114295
8,-1.715617,-0.942177,0.494220,-1.476432,-3.000000,2.272514,1.840807,-1.341816,-3.000000,-0.545940
9,-0.748705,0.142168,2.097138,-3.000000,-3.000000,0.383369,2.575458,0.966087,-2.609528,1.072205


In [156]:
def genetic_algorithm_binary(domain, function):
    """Runs the Genetic Algorithm with binary representation."""
    evaluations = 0
    population = generate_initial_population_binary()
    decoded_population = decode_solution_binary(population, domain)
    fitness_values = np.array([function(decoded_population[i], domain) for i in range(POPULATION_SIZE)])
    evaluations += len(population)

    while evaluations < EVAL_LIMIT:
        # Selection
        parents = [tournament_selection(population, fitness_values, k=2) for _ in range(POPULATION_SIZE)]

        # Recombination
        offspring = []
        for i in range(POPULATION_SIZE // 2):
            child1 = uniform_crossover_binary(parents[2 * i], parents[2 * i + 1])
            child2 = uniform_crossover_binary(parents[2 * i + 1], parents[2 * i])
            offspring.extend([child1, child2])

        # Mutation
        offspring = [binary_mutation(ind) for ind in offspring]

        # Decode and Evaluate Offspring
        decoded_offspring = decode_solution_binary(np.array(offspring), domain)
        offspring_fitness = np.array([function(decoded_offspring[i], domain) for i in range(len(offspring))])
        evaluations += len(offspring)

        # Replacement (generational replacement)
        population = np.array(offspring)
        fitness_values = offspring_fitness

    best_solution = population[np.argmin(fitness_values)]
    best_fitness = np.min(fitness_values)
    return best_solution, best_fitness


In [157]:
def run_simulation_binary_f1():
    """Run binary GA with F1 and return raw data and mean fitness."""
    raw_data = []
    for _ in range(REPEATS):
        _, best_fitness = genetic_algorithm_binary(DOMAIN_1, evaluate_f1)
        raw_data.append(best_fitness)
    return np.array(raw_data)

def run_simulation_real_f1():
    """Run real-valued GA with F1 and return raw data and mean fitness."""
    raw_data = []
    for _ in range(REPEATS):
        _, best_fitness = genetic_algorithm(DOMAIN_1, evaluate_f1)
        raw_data.append(best_fitness)
    return np.array(raw_data)

def run_simulation_binary_f2():
    """Run binary GA with F2 and return raw data and mean fitness."""
    raw_data = []
    for _ in range(REPEATS):
        _, best_fitness = genetic_algorithm_binary(DOMAIN_2, evaluate_f2)
        raw_data.append(best_fitness)
    return np.array(raw_data)

def run_simulation_real_f2():
    """Run real-valued GA with F2 and return raw data and mean fitness."""
    raw_data = []
    for _ in range(REPEATS):
        _, best_fitness = genetic_algorithm(DOMAIN_2, evaluate_f2)
        raw_data.append(best_fitness)
    return np.array(raw_data)

In [158]:
# Main Data Collection
data_binary_f1 = run_simulation_binary_f1()
data_real_f1 = run_simulation_real_f1()
data_binary_f2 = run_simulation_binary_f2()
data_real_f2 = run_simulation_real_f2()

# Prepare Raw Data for Excel
data_dict = {
    "Binary F1": data_binary_f1,
    "Real F1": data_real_f1,
    "Binary F2": data_binary_f2,
    "Real F2": data_real_f2
}

# Write to Excel
excel_filename = "GA_Results.xlsx"
with pd.ExcelWriter(excel_filename) as writer:
    for key, data in data_dict.items():
        df = pd.DataFrame(data)
        df.to_excel(writer, sheet_name=key, index=False)

# Generate Mean Plots
for key, data in data_dict.items():
    plt.figure(figsize=(10, 6))
    plt.plot(data.mean(axis=0), label=f"{key} Mean")
    plt.title(f"Mean Fitness Over Runs ({key})")
    plt.xlabel("Generations")
    plt.ylabel("Fitness")
    plt.legend()
    plt.grid()
    plt.savefig(f"{key}_mean_plot.png")
    plt.close()

print(f"Results saved to {excel_filename}")

Results saved to GA_Results.xlsx
